In [305]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
import re
nltk.download('stopwords')
nltk.download('punkt')
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from scipy.spatial.distance import cosine
!pip install gensim
from gensim.models import Word2Vec
import numpy as np
import gensim.downloader as api
path = api.load("word2vec-google-news-300", return_path=True)
from gensim.models import KeyedVectors


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
dataset=pd.read_csv('full_id.csv')
dataset.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,sarcasm,text,label,emotion,id
0,0,Didn't have to tell you that Somebody's a Jan...,0,anger,1
1,0,already respects the bucket.,0,anger,2
2,0,"I accept all those flaws, why can't you accep...",0,anger,3
3,0,"This is my wife Nancy. Look, get out!",0,anger,4
4,0,Fine! Then you tell Roger because he was real...,0,anger,5


In [308]:
dataset['text'] = dataset['text'].apply(lambda x: x.lower())

In [309]:
def clean_and_tokenize_STOP(text):
    stop_words = set(stopwords.words('english'))
    # Remove commas, periods, question marks, and exclamation marks
    text = re.sub(r'[.,?!\{}()、—·…]', '', text)
    # Tokenize by splitting at spaces
    tokens = text.split(" ")
    # Convert tokens to lowercase
    tokens = [token.lower() for token in tokens if token and token.lower() not in stop_words]
    return tokens
def clean_and_tokenize(text):
    # Remove commas, periods, question marks, and exclamation marks
    text = re.sub(r'[.,?!\{}()、—·…]', '', text)
    # Tokenize by splitting at spaces
    tokens = text.split(" ")
    # Convert tokens to lowercase
    tokens = [token.lower() for token in tokens if token]
    return tokens

# Apply function to the "text" column in the dataset
dataset["tokenized_without_stop"] = dataset["text"].apply(clean_and_tokenize_STOP)
dataset["tokenized_with_stop"] = dataset["text"].apply(clean_and_tokenize)

In [312]:
# Load pre-trained BERT model and tokenizer
model = BertForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
tokenizer = BertTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

def clean_and_split_sentence(text):
    # Strip spaces, replace ? and ! with empty string
    text = text.strip(" ")
    text = re.sub(r'[?!]', '', text)

    # Step 2: If period is at the end, don't split it
    if text.endswith('.'):
        # Remove the period for splitting
        text_without_period = text[:-1]
        split_text = re.split(r'[,.]', text_without_period)
        # Remove the period from the last element if it was added back
        split_text[-1] = split_text[-1].strip('.')
    else:
        split_text = re.split(r'[,.]', text)

    return [i.strip() for i in split_text if i != ""]

# Apply the function to the 'text' column
dataset['sentence_split'] = dataset['text'].apply(clean_and_split_sentence)


def sentence_level_similarity_emotion(sentences, tokenizer, model):
    index_pairs = [(i, i+1) for i in range(len(sentences)-1)]
    similarity = []

    if len(index_pairs) != 0:
        for i in index_pairs:
            sentence_1 = sentences[i[0]]
            sentence_2 = sentences[i[1]]

            with torch.no_grad():
                # Get embeddings for sentence 1
                input_1 = tokenizer(sentence_1, return_tensors="pt", truncation=True, padding=True, max_length=512)
                outputs_1 = model(**input_1)
                embedding_1 = outputs_1.logits.squeeze().numpy()

                # Get embeddings for sentence 2
                input_2 = tokenizer(sentence_2, return_tensors='pt', truncation=True, padding=True, max_length=512)
                outputs_2 = model(**input_2)
                embedding_2 = outputs_2.logits.squeeze().numpy()

                # Calculate cosine similarity and append to the list
                similarity += [1 - cosine(embedding_1, embedding_2)]
    else:
        similarity += [0]

    # Return the average similarity score
    return np.mean(similarity)

dataset['sentence_level_similarity_emotion'] = dataset['sentence_split'].apply(lambda x: sentence_level_similarity_emotion(x, tokenizer, model))

In [315]:
# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Define the function to calculate sentence-level similarity
def sentence_level_similarity_word(sentences):
    sentences = [(" ").join(clean_and_tokenize_STOP(i)).strip() for i in sentences]
    index_pairs = [(i, i+1) for i in range(len(sentences)-1)]
    similarity = []

    if len(index_pairs) != 0:
        for i in index_pairs:
            sentence_1 = sentences[i[0]]
            sentence_2 = sentences[i[1]]

            with torch.no_grad():
                # Get embeddings for sentence 1
                input_1 = tokenizer(sentence_1, return_tensors="pt", truncation=True, padding=True, max_length=512)
                outputs_1 = model(**input_1)
                embedding_1 = outputs_1.last_hidden_state[:, 0, :].squeeze().numpy()  # CLS token embedding

                # Get embeddings for sentence 2
                input_2 = tokenizer(sentence_2, return_tensors='pt', truncation=True, padding=True, max_length=512)
                outputs_2 = model(**input_2)
                embedding_2 = outputs_2.last_hidden_state[:, 0, :].squeeze().numpy()  # CLS token embedding

                # Calculate cosine similarity between embeddings
                similarity += [1 - cosine(embedding_1, embedding_2)]
    else:
        similarity += [0]

    # Return the average similarity score
    return np.mean(similarity)

dataset['sentence_level_similarity_word'] = dataset['sentence_split'].apply(sentence_level_similarity_word)

In [317]:
dataset['exclamation'] = dataset['text'].str.contains('!')

In [318]:
model = KeyedVectors.load_word2vec_format(path, binary=True)

# Average word vectors to create sentence embeddings
def sentence_to_vector(words):
    word_vectors = [model[word] for word in words if word in model]
    return np.mean(word_vectors, axis=0) if word_vectors else np.zeros(300)
dataset['word2vec_pretrained_with_stop'] = dataset['tokenized_with_stop'].apply(sentence_to_vector)
dataset['word2vec_pretrained_without_stop'] = dataset['tokenized_without_stop'].apply(sentence_to_vector)

In [322]:
dataset.to_csv("text_feature.csv")